# Motivation
Let's say we're building a local editor that allows you to load an AIConfig
from a local file and then run methods on it.

In the (simplified) code below, we do just that.

In [16]:
import json
from typing import Any


def read_json_from_file(path: str) -> dict[str, Any]:
    with open(path, "r") as f:
        return json.loads(f.read())
    

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    print(f"Loaded AIConfig: {aiconfig['name']}\n")


start_app("cookbooks/Getting-Started/travel.aiconfig.json")

Loaded AIConfig: NYC Trip Planner




## Cool, LGTM, ship it!

# A few hours later...

# Issue #9000 Editor crashes on new file path
### opened 2 hours ago by lastmile-biggest-fan

Dear LastMile team,
I really like the editor, but when I give it a new file path, it crashes!
I was hoping it would create a new AIConfig for me and write it to the file...

# OK, what happened?

In [17]:
start_app("i-dont-exist-yet-please-create-me.json")

FileNotFoundError: [Errno 2] No such file or directory: 'i-dont-exist-yet-please-create-me.json'

# Oops

Ok, let's diagnose the problem here. We forgot to handle the case where the path doesn't exist.

That's understandable. As programmers, we don't always write perfect code.
Sometimes it's helpful to bring new tools into the workflow to prevent this kind of problem in the future.


Hmm, ok. Wouldn't it be nice if we had a static analyzer that could have caught this problem immediately? That way we could have fixed it before the initial PR was merged.

Let's analyze some tools.

## V2: Optional

In [6]:
from typing import Any, Optional


def read_json_from_file(path: str) -> Optional[dict[str, Any]]:
    with open(path, "r") as f:
        return json.loads(f.read())
    

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    print(f"Loaded AIConfig: {aiconfig['name']}\n")

print("""
[Pyright] Object of type "None" is not subscriptable
PylancereportOptionalSubscript
(variable) aiconfig: dict[str, Any] | None
""")


[Pyright] Object of type "None" is not subscriptable
PylancereportOptionalSubscript
(variable) aiconfig: dict[str, Any] | None



# Aha!

Now, Pyright immediately tells us that `None` is a possibility, and we have to handle this case. Let's do that.


In [31]:
from typing import Optional
from aiconfig.Config import AIConfigRuntime



def read_json_from_file(path: str) -> Optional[dict[str, Any]]:
    try:
        with open(path, "r") as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return None

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    if aiconfig is None:
        print(f"Could not load AIConfig from path: {path}. Creating and saving.")
        aiconfig = json.dumps(AIConfigRuntime.create())
        # [save the aiconfig to the path]        
        print(f"Loaded and saved new AIConfig\n")
    else:
        print(f"Loaded AIConfig: {aiconfig}\n")

start_app("cookbooks/Getting-Started/travel.aiconfig.json")
start_app("i-dont-exist-yet-please-create-me.json")

Loaded AIConfig: NYC Trip Planner

Loaded AIConfig: 



Ok, cool, much better. But wait, it would be nice to retain some information about what went wrong. My `None` value doesn't tell me anything about why the AIConfig couldn't be loaded. Does the file not exist? Was it a permission problem, networked filesystem problem? etc.

# V3: Result

The result library (https://github.com/rustedpy/result) provides a neat type
called `Result`, which is a bit like Optional. It's parametrized by the value type just like optional, but also by a second type for the error case.

We can use it like optional, but store an arbitrary value with information about what went wrong.

In [52]:
from aiconfig.Config import AIConfigRuntime
from result import Result, Ok, Err
from typing import Any

from json import JSONDecodeError


def read_json_from_file(path: str) -> Result[dict[str, Any], str]:
    """Use `str` in the error case to contain a helpful error message."""
    try:
        with open(path, "r") as f:
            return Ok(json.loads(f.read()))
    except FileNotFoundError:
        return Err(f"File not found at path: {path}")
    except OSError as e:
        return Err(f"Could not read file at path: {path}: {e}")
    except JSONDecodeError as e:
        return Err(f"Could not parse JSON at path: {path}: {e}")

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    file_contents = read_json_from_file(path)
    match file_contents:
        case Ok(aiconfig_ok):
            print(f"Loaded AIConfig: {aiconfig_ok['name']}\n")
        case Err(e):
            print(f"Could not load AIConfig from path: {path} ({e}). Creating and saving.")
            aiconfig = AIConfigRuntime.create().model_dump(exclude="callback_manager")
            # [Save to file path]
            # aiconfig.save(path)
            print(f"Created and saved new AIConfig: {aiconfig['name']}\n")

start_app("cookbooks/Getting-Started/travel.aiconfig.json")
start_app("i-dont-exist-yet-please-create-me.json")

Loaded AIConfig: NYC Trip Planner

Could not load AIConfig from path: i-dont-exist-yet-please-create-me.json (File not found at path: i-dont-exist-yet-please-create-me.json). Creating and saving.
Created and saved new AIConfig: 



There are several nice things about this pattern:
* You get static errors similar to the Optional case unless you check for None
* You also get specific, useful error information
* Structural pattern matching: When matching the cases, you can elegantly and safely unbox the data inside the result.
* Because of pyright's ability to check for exhaustive pattern matching, it will yell at you if you don't handle the Err case. Try it! Comment out the Err case.

# Part 2: Composition (To be continued)